In [48]:
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv

import time
#!pip install tenacity
from tenacity import retry, stop_after_attempt, wait_exponential

#!pip install anthropic
import anthropic

load_dotenv()

True

In [38]:
os.chdir(r"C:\Users\aeveg\OneDrive\Documentos\MASTERS\Proyecto\Proyecto de Materia\ProyectoDesarrolloSoluciones")

In [39]:
def load_imdb_reviews(ruta_archivo):
    # Lee el archivo CSV y lo guarda en un DataFrame
    df = pd.read_csv(ruta_archivo)    
    return df

In [40]:
df_train = load_imdb_reviews('./data/imdb_reviews_train.csv')
print("\nPrimeras 5 filas del dataset de entrenamiento de reseñas de IMDB:")
df_train.head()


Primeras 5 filas del dataset de entrenamiento de reseñas de IMDB:


,review,sentiment
0,In Panic In The Streets Richard Widmark plays ...,1
1,If you ask me the first one was really better ...,0
2,I am a big fan a Faerie Tale Theatre and I've ...,1
3,I just finished reading a book about Dillinger...,0
4,Greg Davis and Bryan Daly take some crazed sta...,0


In [41]:
df_test = load_imdb_reviews('./data/imdb_reviews_test.csv')
print("\nPrimeras 5 filas del dataset de pruebas de reseñas de IMDB:")
df_test.head()


Primeras 5 filas del dataset de pruebas de reseñas de IMDB:


,review,sentiment
0,"When I was a kid, I loved ""Tiny Toons"". I espe...",1
1,"The setup for ""Nature of the Beast"" is ingenio...",0
2,I do not have much to say than this is a great...,1
3,Extremely formulaic with cosmic-sized logic ho...,0
4,I actually liked certain things about this gam...,0


In [ ]:
@retry(stop=stop_after_attempt(5), 
       wait=wait_exponential(multiplier=1, min=4, max=60),
       before_sleep=lambda retry_state: print(f"Intento de traducción {retry_state.attempt_number} falló, reintentando en {retry_state.next_action.sleep} segundos..."))
def translate_single_review(client, review):
    prompt = f"Translate the following English review to Spanish. Only return the translation, nothing else: {review}"
    
    message = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

def traducir_reviews(dataframe):
    # Create new empty DataFrame with same columns 
    df_translated = pd.DataFrame(columns=['review', 'sentiment'])

    client = anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    # Iterate through original DataFrame
    for index, row in dataframe.iterrows():
        if index % 100 == 0:
            print(f"Traduciendo reseña {index+1} de {len(dataframe)}")
            
        try:
            # Try to translate with retries
            translated_review = translate_single_review(client, row['review'])
            
            # Add new row to translated DataFrame
            new_row = pd.DataFrame({
                'review': [translated_review],
                'sentiment': [row['sentiment']]
            })
            df_translated = pd.concat([df_translated, new_row], ignore_index=True)
            
        except Exception as e:
            print(f"Error al traducir reseña {index+1}: {str(e)}")
            
    return df_translated

In [51]:
df_train_translated = traducir_reviews(df_train[:5])
print(df_train_translated.head())

Traduciendo reseña 1 de 5
                                              review sentiment
0  En Pánico en las Calles, Richard Widmark inter...         1
1  Si me preguntas, la primera era realmente mejo...         0
2  Soy un gran fan de Faerie Tale Theatre y los h...         1
3  Acabo de terminar de leer un libro sobre Dilli...         0
4  Greg Davis y Bryan Daly toman algunas declarac...         0


In [52]:
df_train_translated.to_csv('data/imdb_reviews_train_espanol.csv', index=False)
print('dataset de entrenamiento traducido a español')

dataset de entrenamiento traducido a español


In [53]:
df_test_translated = traducir_reviews(df_test[:5])
print(df_test_translated.head())

Traduciendo reseña 1 de 5
                                              review sentiment
0  Cuando era niño, me encantaban los "Tiny Toons...         1
1  La configuración de "Nature of the Beast" es i...         0
2  No tengo mucho que decir más que este es un gr...         1
3  Extremadamente formulaico con agujeros lógicos...         0
4  En realidad me gustaron ciertas cosas de este ...         0


In [54]:
df_test_translated.to_csv('data/imdb_reviews_test_espanol.csv', index=False)
print('dataset de pruebas traducido a español')

dataset de pruebas traducido a español
